In [7]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
import gradio as gr

# Load BERT Model and Tokenizer
bert_model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
bert_tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

# Load RoBERTa Model and Tokenizer
roberta_model = AutoModelForQuestionAnswering.from_pretrained("deepset/roberta-base-squad2")
roberta_tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2")

# Create pipelines for inference
bert_pipeline = pipeline("question-answering", model=bert_model, tokenizer=bert_tokenizer)
roberta_pipeline = pipeline("question-answering", model=roberta_model, tokenizer=roberta_tokenizer)

def evaluate_question(context, question, model_choice):
    if model_choice == "BERT":
        answer = bert_pipeline(question=question, context=context)
    elif model_choice == "RoBERTa":
        answer = roberta_pipeline(question=question, context=context)
    else:
        answer = {"answer": "Invalid model choice"}
    
    return f"Answer: {answer['answer']}\nConfidence: {answer['score']:.2f}"


# Define Gradio Interface
def qa_interface(context, question, model_choice):
    return evaluate_question(context, question, model_choice)

interface = gr.Interface(
    fn=qa_interface,
    inputs=[
        gr.Textbox(label="Context", lines=5, placeholder="Enter context here..."),
        gr.Textbox(label="Question", placeholder="Enter your question here..."),
        gr.Radio(choices=["BERT", "RoBERTa"], label="Select Model")
    ],
    outputs=gr.Textbox(label="Answer")
)

interface.launch()
import pandas as pd
from datasets import load_dataset
import evaluate
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline, TrainingArguments, Trainer

# Load dataset and metric
dataset = load_dataset("squad_v2")
metric = evaluate.load("squad")

# Load Models and Tokenizer
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

# Preprocess Function
def preprocess_function(examples):
    return tokenizer(
        examples["question"], examples["context"], truncation=True, padding="max_length", max_length=384
    )

# Tokenize Dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"]
)

# Train the Model
trainer.train()

# Save the Trained Model
trainer.save_model("./fine_tuned_model")


# Save the fine-tuned model
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

# Load the fine-tuned model
fine_tuned_model = AutoModelForQuestionAnswering.from_pretrained("./fine_tuned_model")
fine_tuned_tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_model")
fine_tuned_pipeline = pipeline("question-answering", model=fine_tuned_model, tokenizer=fine_tuned_tokenizer)

from datasets import load_metric

# Define metric
metric = load_metric("squad_v2")

# Evaluate
def evaluate_model(model_pipeline, dataset):
    results = []
    for example in dataset:
        prediction = model_pipeline(question=example["question"], context=example["context"])
        results.append({
            "prediction_text": prediction["answer"],
            "id": example["id"],
            "references": [{"text": example["answers"]["text"][0], "id": example["id"]}]
        })
    return metric.compute(predictions=results, references=results)

bert_results = evaluate_model(bert_pipeline, dataset["validation"])
roberta_results = evaluate_model(roberta_pipeline, dataset["validation"])
print("BERT Results:", bert_results)
print("RoBERTa Results:", roberta_results)



Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps:0
Device set to use mps:0


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Map:   0%|          | 0/130319 [00:00<?, ? examples/s]

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

/opt/anaconda3/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>={ACCELERATE_MIN_VERSION}'`

In [3]:
pip install gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 23.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.7/248.7 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 32.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 20.6 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing installation: tomlkit 0.11.1
    Uninstalling tomlkit-0.11.1:
      Successfully uninstalled tomlkit-0.11.1
Note: you may need to restar